# Input libraries

In [1]:
from IPython.core.interactiveshell import InteractiveShell
import glob
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import geopandas as gpd
import geoparquet as gpq
import math
from shapely.geometry import Point

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import dask_geopandas as dg


from pathlib import Path
import os
import glob

import re
from tqdm.notebook import tqdm

%matplotlib inline
InteractiveShell.ast_node_interactivity = "all"

pbar = ProgressBar()
pbar.register()

ModuleNotFoundError: No module named 'geoparquet'

# Input data

### sa data

In [0]:
sa = gpd.read_file('data/shapefile/Finn_SA2Summary.shp')
sa.to_crs("EPSG:2193")
sa = sa[['BldgFloo_1', 'PopDensity', 'NumberBldg', 'geometry']] # drop column not used in caluclation of Ignition
sa = gpd.GeoDataFrame(sa)

### pga data

In [0]:
pga = pd.read_parquet('/Users/alex/Dropbox/Work/Repository/OpenQuake/output/PGAxy.parquet')
pga = gpd.GeoDataFrame(pga, geometry=gpd.points_from_xy(pga.lon, pga.lat, crs="EPSG:2193"))

# Merge SA data with PGA events 

In [0]:
PGA_SA = gpd.sjoin(pga, sa, op="within")

# Calculate Ignition probability

In [18]:
test = PGA_SA.head(5)

test

In [38]:
test['SA_IgnPrb'] = 1/(np.exp(-1*(-6.755+8.463+test['gmv_PGA']+0.0000984+test['PopDensity']+0.001523+test['BldgFloo_1'])))
test['IgnProb_bl'] = 1 - np.power(1-test['SA_IgnPrb'], +1/test['NumberBldg'])

/Users/alex/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [39]:
test

,event_id,site_id,gmv_PGA,lon,lat,geometry,index_right,BldgFloo_1,PopDensity,NumberBldg,SA_IgnPrb,IgnProb,ignA,IgnProb_bl
__null_dask_index__,,,,,,,,,,,,,,
0,0,0,0.050537,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,0.21756,2208,1290,inf,-inf,inf,-inf
76,1,0,0.259768,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,0.21756,2208,1290,inf,-inf,inf,-inf
152,2,0,0.099314,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,0.21756,2208,1290,inf,-inf,inf,-inf
228,3,0,0.128333,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,0.21756,2208,1290,inf,-inf,inf,-inf
304,4,0,0.017772,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,0.21756,2208,1290,inf,-inf,inf,-inf


In [20]:
def SA_Ignition_Probability(merge_data):
    merge_data['SA_IgnPrb'] = 1/(np.exp(-1*(-6.755+8.463+merge_data['gmv_PGA'] + 0.0000984 + merge_data['PopDensity']+ 0.001523+merge_data['BldgFloo_1'])))
    merge_data['IgnProb'] = 1 - np.power(1-merge_data['SA_IgnPrb'], + 1/merge_data['NumberBldg'])
    return merge_data

In [35]:
test['ignA']= np.exp(-1*(-6.755+8.463)+test['gmv_PGA'] 
                     + 0.0000984 
                     + test['PopDensity'])
#                     + test['BldgFloo_1'])
test['ignA']

/Users/alex/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/alex/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


__null_dask_index__
0      inf
76     inf
152    inf
228    inf
304    inf
Name: ignA, dtype: float64

In [24]:
IgnitionData = SA_Ignition_Probability(test)

/Users/alex/anaconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


# Save to file

In [22]:
IgnitionData.drop(['lon', 'lat', 'index_right', \
                   'BldgFloo_1', 'PopDensity', 'NumberBldg'], \
                  axis=1, \
                 inplace=True)

In [25]:
IgnitionData.head()

,event_id,site_id,gmv_PGA,lon,lat,geometry,index_right,BldgFloo_1,PopDensity,NumberBldg,SA_IgnPrb,IgnProb
__null_dask_index__,,,,,,,,,,,,
0,0,0,0.050537,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,0.21756,2208,1290,inf,-inf
76,1,0,0.259768,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,0.21756,2208,1290,inf,-inf
152,2,0,0.099314,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,0.21756,2208,1290,inf,-inf
228,3,0,0.128333,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,0.21756,2208,1290,inf,-inf
304,4,0,0.017772,1.752780e+06,5.441405e+06,POINT (1752779.830 5441405.376),0,0.21756,2208,1290,inf,-inf


In [10]:
p_event = Path('/Users/alex/Dropbox/Work/GNS/008_FFE/ProbaFFE/PGA_event')

for event in range(1001):#(IgnitionData['event_id'].max()):
    print(f"event {event}")
        i_data = IgnitionData[IgnitionData['event_id']==event]
        i_data.to_file(str(p_event) + f"/event_{event}.shp")
#     try:
#         print(f"event {event}")
#         i_data = IgnitionData[IgnitionData['event_id']==event]
#         i_data.to_file(str(p_event) + f"/event_{event}.shp")
#     except:
#         print(f"no event {event}")

event 0


/Users/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  import sys


no event 0
event 1
no event 1
event 2
no event 2
event 3
no event 3
event 4
no event 4
event 5
no event 5
event 6
no event 6
event 7
no event 7
event 8
no event 8
event 9
no event 9
event 10
no event 10
event 11
no event 11
event 12
no event 12
event 13
no event 13
event 14
no event 14
event 15
no event 15
event 16
no event 16
event 17
no event 17
event 18
no event 18
event 19
no event 19
event 20
no event 20
event 21
no event 21
event 22
no event 22
event 23
no event 23
event 24
no event 24
event 25
no event 25
event 26
no event 26
event 27
no event 27
event 28
no event 28
event 29
no event 29
event 30
no event 30
event 31
no event 31
event 32
no event 32
event 33
no event 33
event 34
no event 34
event 35
no event 35
event 36
no event 36
event 37
no event 37
event 38
no event 38
event 39
no event 39
event 40
no event 40
event 41
no event 41
event 42
no event 42
event 43
no event 43
event 44
no event 44
event 45
no event 45
event 46
no event 46
event 47
no event 47
event 48
no event 48

no event 368
event 369
no event 369
event 370
no event 370
event 371
no event 371
event 372
no event 372
event 373
no event 373
event 374
no event 374
event 375
no event 375
event 376
no event 376
event 377
no event 377
event 378
no event 378
event 379
no event 379
event 380
no event 380
event 381
no event 381
event 382
no event 382
event 383
no event 383
event 384
no event 384
event 385
no event 385
event 386
no event 386
event 387
no event 387
event 388
no event 388
event 389
no event 389
event 390
no event 390
event 391
no event 391
event 392
no event 392
event 393
no event 393
event 394
no event 394
event 395
no event 395
event 396
no event 396
event 397
no event 397
event 398
no event 398
event 399
no event 399
event 400
no event 400
event 401
no event 401
event 402
no event 402
event 403
no event 403
event 404
no event 404
event 405
no event 405
event 406
no event 406
event 407
no event 407
event 408
no event 408
event 409
no event 409
event 410
no event 410
event 411
no event 41

no event 727
event 728
no event 728
event 729
no event 729
event 730
no event 730
event 731
no event 731
event 732
no event 732
event 733
no event 733
event 734
no event 734
event 735
no event 735
event 736
no event 736
event 737
no event 737
event 738
no event 738
event 739
no event 739
event 740
no event 740
event 741
no event 741
event 742
no event 742
event 743
no event 743
event 744
no event 744
event 745
no event 745
event 746
no event 746
event 747
no event 747
event 748
no event 748
event 749
no event 749
event 750
no event 750
event 751
no event 751
event 752
no event 752
event 753
no event 753
event 754
no event 754
event 755
no event 755
event 756
no event 756
event 757
no event 757
event 758
no event 758
event 759
no event 759
event 760
no event 760
event 761
no event 761
event 762
no event 762
event 763
no event 763
event 764
no event 764
event 765
no event 765
event 766
no event 766
event 767
no event 767
event 768
no event 768
event 769
no event 769
event 770
no event 77

In [2]:
!du -sh /Users/alex/Dropbox/Work/GNS/008_FFE/ProbaFFE/PGA_event

1.6G	/Users/alex/Dropbox/Work/GNS/008_FFE/ProbaFFE/PGA_event
